# scratch work

In [10]:
import jax

jax.config.update("jax_enable_x64", True)
import astropy.units as u
import jax.numpy as jnp
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astroquery.jplhorizons import Horizons

from jorbit import Observations, Particle


In [11]:
nights = [Time("2025-01-01 07:00"), Time("2025-01-02 07:00"), Time("2025-01-05 07:00")]

times = []
for n in nights:
    times.extend([n + i * 1 * u.hour for i in range(3)])
times = Time(times)


# obj = Horizons(id="274301", location="695@399", epochs=times.utc.jd)
obj = Horizons(id="90377", location="695@399", epochs=times.utc.jd)
pts = obj.ephemerides(extra_precision=True, quantities="1")

coords = SkyCoord(pts["RA"], pts["DEC"], unit=(u.deg, u.deg))
times = Time(pts["datetime_jd"], format="jd", scale="utc")

In [12]:
obs = Observations(
    observed_coordinates=coords,
    times=times,
    observatories="kitt peak",
    astrometric_uncertainties=1 * u.arcsec,
)
obs

Observations with 9 set(s) of observations

In [13]:
from jorbit.astrometry.orbit_fit_seeds import gauss_method_orbit

gauss_method_orbit(obs[0] + obs[1] + obs[-1]).to_keplerian()

KeplerianState(semi=Array([532.05169735], dtype=float64), ecc=Array([0.85738742], dtype=float64), inc=Array([11.92849799], dtype=float64), Omega=Array([144.40266084], dtype=float64), omega=Array([311.99163074], dtype=float64), nu=Array([323.84396056], dtype=float64), time=Array(2460676.79246741, dtype=float64))

In [14]:
# obj = Horizons(id="274301", location="500@0", epochs=times.tdb.jd[0])
obj = Horizons(id="90377", location="500@0", epochs=times.tdb.jd[0])
vecs = obj.vectors(refplane="earth")
true_x0 = jnp.array([vecs["x"], vecs["y"], vecs["z"]]).T[0]
true_v0 = jnp.array([vecs["vx"], vecs["vy"], vecs["vz"]]).T[0]
true_x0, true_v0

(Array([40.298248  , 71.80844307, 12.46825354], dtype=float64),
 Array([-0.00247257,  0.00048492,  0.00042657], dtype=float64))

In [15]:
# p0 = Particle(
#     x=true_x0, v=true_v0, time=times[0], name="274301 Wikipedia", observations=obs
# )
p0 = Particle(
    x=true_x0, v=true_v0, time=times[0], name="90377 Sedna", observations=obs,
)
p0

Particle: 90377 Sedna

In [16]:
p1 = p0.max_likelihood(verbose=True)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            6     M =          100

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.47971D+01    |proj g|=  6.96136D+04


 This problem is unconstrained.



At iterate    1    f=  3.21828D+01    |proj g|=  2.31412D+04

At iterate    2    f=  2.80098D+01    |proj g|=  1.98105D+04

At iterate    3    f=  1.65409D+01    |proj g|=  6.22540D-03

At iterate    4    f=  1.65409D+01    |proj g|=  3.49577D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    6      4      7      1     0     0   3.496D-05   1.654D+01
  F =   16.540893597898432     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


In [17]:
p1.residuals(p1.cartesian_state), p0.residuals(p0.cartesian_state)

(Array([[-8.09618597e-06, -2.05621338e-06],
        [-6.73160907e-06, -1.25850821e-06],
        [-5.46306045e-06, -1.66085966e-06],
        [ 8.92576989e-06,  2.26832021e-06],
        [ 9.35403645e-06,  2.08494868e-06],
        [ 8.80070142e-06,  2.26148648e-06],
        [-2.44636998e-06, -1.97665041e-07],
        [-1.40586478e-06,  3.80336955e-07],
        [-2.94478315e-06, -1.82466616e-06]], dtype=float64),
 Array([[-1.47200117e-06,  1.07532008e-06],
        [-1.18933989e-06,  1.49255446e-06],
        [-9.10482747e-07,  7.05574422e-07],
        [ 1.01598318e-06, -9.71389347e-07],
        [ 7.29844464e-07, -1.38023574e-06],
        [-4.38202549e-07, -1.43192019e-06],
        [-1.74874341e-07, -9.56226198e-08],
        [ 1.30691672e-06,  7.22190964e-07],
        [ 3.07147170e-07, -1.24653535e-06]], dtype=float64))

In [20]:
print(p0.cartesian_state)
print(p0.keplerian_state)

CartesianState(x=Array([[40.298248  , 71.80844307, 12.46825354]], dtype=float64), v=Array([[-0.00247257,  0.00048492,  0.00042657]], dtype=float64), time=np.float64(2460676.792467407))
KeplerianState(semi=Array([506.419829], dtype=float64), ecc=Array([0.84954883], dtype=float64), inc=Array([11.92852394], dtype=float64), Omega=Array([144.40149979], dtype=float64), omega=Array([311.28485567], dtype=float64), nu=Array([324.55266997], dtype=float64), time=np.float64(2460676.792467407))


In [22]:
p0.residuals(p0.cartesian_state)

Array([[-1.47200117e-06,  1.07532008e-06],
       [-1.18933989e-06,  1.49255446e-06],
       [-9.10482747e-07,  7.05574422e-07],
       [ 1.01598318e-06, -9.71389347e-07],
       [ 7.29844464e-07, -1.38023574e-06],
       [-4.38202549e-07, -1.43192019e-06],
       [-1.74874341e-07, -9.56226198e-08],
       [ 1.30691672e-06,  7.22190964e-07],
       [ 3.07147170e-07, -1.24653535e-06]], dtype=float64)

In [9]:
p0.loglike(p0.cartesian_state)

Array(-16.5408936, dtype=float64)

In [10]:
jax.grad(p0.loglike)(p0.cartesian_state)

CartesianState(x=Array([[-0.17490244, -0.31958406, -0.21653323]], dtype=float64), v=Array([[-0.34285022, -0.64230222, -0.30679037]], dtype=float64), time=Array(-0.00342397, dtype=float64))

In [11]:
x = jnp.concatenate([p0.cartesian_state.x.flatten(), p0.cartesian_state.v.flatten()])
p0.scipy_objective(x / p0.fit_scales)

Array(16.5408936, dtype=float64)

In [12]:
x = jnp.concatenate([p0.cartesian_state.x.flatten(), p0.cartesian_state.v.flatten()])
p0.scipy_objective_grad(x / p0.fit_scales)

Array([-0.3508416 ,  0.56848201,  0.11255093, -0.00226829, -0.00428758,
       -0.00063009], dtype=float64)

In [13]:
x / p0.fit_scales

Array([0.99989828, 0.99987798, 0.99991297, 1.00663733, 0.9930172 ,
       0.99271124], dtype=float64)